In [3]:
import psycopg2 as ps
import pandas as pd
import altair as alt

Load data from database

In [23]:
conn = ps.connect(database="example", user="user", password="user_pass", host="picasso.experiments.cs.kuleuven.be", port="3895")
query = "SELECT * FROM top10"
df = pd.read_sql_query(query, conn)
conn.close()
questionnaire = pd.read_csv('questionnaire_parsed.csv')
# questionnaire.head()
#join isExpert column to df by matching userid
df = df.merge(questionnaire[['userid','isExpert']], on='userid', how='left')


colpak = []
for entry in df['topx']:
    count = 0
    for i in entry:
        if i['real_rank'] <= 6:
            count += 1

    pak = count/6
    colpak.append(pak)
df['pak'] = colpak
df["version"] = df["version"].apply(lambda x: "LC + EH" if x == 1 else "LC + ES" if x == 2 else "HC + EH" if x == 3 else "HC + ES")
#get first entry
df


C:\Users\morti\AppData\Local\Temp\ipykernel_4984\3473058840.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,id,userid,version,poseweight,colorweight,objectweight,topx,pak,src,mse,duration,isExpert
0,10,df3aaac2c129,LC + EH,0.000000,1.000000,0.000000,"[{'image_name': 'images_1373.jpg', 'personal_r...",0.166667,-0.085714,1483.666667,56,Non-Expert
1,12,04a419bf11412,LC + ES,0.000000,1.000000,1.000000,"[{'image_name': 'images_1085.jpg', 'personal_r...",0.666667,0.828571,5.000000,96,Non-Expert
2,12,04a419bf11412,LC + ES,0.000000,1.000000,1.000000,"[{'image_name': 'images_1085.jpg', 'personal_r...",0.666667,0.828571,5.000000,96,Non-Expert
3,13,e6c309da2bc4f,LC + EH,1.000000,1.000000,1.000000,"[{'image_name': 'Jean-Beraud_Au-Café.jpg', 'pe...",0.500000,0.885714,437.666667,291,Non-Expert
4,9,df3aaac2c129,LC + EH,0.000000,0.825549,1.000000,"[{'image_name': 'images_1319.jpg', 'personal_r...",0.166667,0.371429,630.500000,173,Non-Expert
...,...,...,...,...,...,...,...,...,...,...,...,...
57,62,85270c2be10b7,LC + ES,1.000000,0.000000,0.515110,"[{'image_name': 'images_2902.jpg', 'personal_r...",0.333333,0.828571,31.500000,267,NaN
58,63,32993a5b624ce,HC + ES,1.000000,1.000000,0.000000,"[{'image_name': 'images_700.jpg', 'personal_ra...",0.500000,0.885714,32.666667,194,Non-Expert
59,64,a6be8c17c479a,LC + EH,0.866758,0.056319,0.888736,"[{'image_name': 'images_1319.jpg', 'personal_r...",0.333333,0.714286,1192.333333,120,Non-Expert
60,6,f00db05d7655b,LC + ES,0.221154,0.303571,0.660714,"[{'image_name': 'images_1319.jpg', 'personal_r...",0.500000,0.771429,174.833333,99,Non-Expert


In [92]:
#calculate average mse, pak and src for each user
df['avg_mse'] = df.groupby('userid')['mse'].transform('mean')
df['avg_pak'] = df.groupby('userid')['pak'].transform('mean')
df['avg_src'] = df.groupby('userid')['src'].transform('mean')


#for every user, calculate average mse, pak and src and put it in a new dataframe
user_df = df[['userid','avg_mse','avg_pak','avg_src']].drop_duplicates()

#save user_df to csv
user_df.to_csv('avg_topX.csv', index=False)

Plot data

In [24]:
#Check if the users are more accurate when they are taking more time


mse  = alt.Chart(df).mark_circle().encode(
    x='duration',
    y='mse',
    color='isExpert',
    tooltip=['version', 'mse', 'duration', 'isExpert']
).properties(
    title='Time vs MSE vs Version vs isExpert'
)

pak = alt.Chart(df).mark_circle().encode(
    x='duration',
    y='pak',
    color='isExpert',
    tooltip=['version', 'pak', 'duration', 'isExpert']
).properties(
    title='Time vs PAK vs Version vs isExpert'
)

src = alt.Chart(df).mark_circle().encode(
    x='duration',
    y='src',
    color='isExpert',
    tooltip=['version', 'src', 'duration', 'isExpert']
).properties(
    title='Time vs SRC vs Version vs isExpert'
)

(mse | pak | src).save('charts/time_vs_metrics.png')

mse | pak | src

alt.HConcatChart(...)

In [31]:
# Chart 1: Average MSE for each version
chart1 = alt.Chart(df).mark_bar().encode(
    x=alt.X('version', sort=['LC + EH', 'LC + ES', 'HC + EH', 'HC + ES']),
    y='mean(mse)'
).properties(
    title='Average Mean Squared Error (MSE) for Each Version',
    width=250,
)

# Chart 2: Average Duration for each version
chart2 = alt.Chart(df).mark_bar().encode(
    x=alt.X('version', sort=['LC + EH', 'LC + ES', 'HC + EH', 'HC + ES']),
    y='mean(duration)',
).properties(
    title='Average Duration for Each Version',
    width=250,
)


(chart1 | chart2).save('charts/mse_vs_duration.png')

(chart1 | chart2)

alt.HConcatChart(...)

Scatter Plot: Performance Metrics Comparison

In [10]:
#check if there is a correlation between mse and pak

chart = alt.Chart(df).mark_circle().encode(
    x='pak:Q',
    y='mse:Q',
    color='version:N',
    tooltip=['version', 'pak', 'src']
)

chart

alt.Chart(...)

In [103]:
# Data preparation

precision_data = df.groupby("version")["pak"].mean().reset_index()

# Bar chart
chart = alt.Chart(precision_data).mark_bar().encode(
    x=alt.X("version", sort=["LC + EH", "LC + ES", "HC + EH", "HC + ES"]),
    y="pak",
    color="version",
    tooltip=["version", "pak"]
).properties(
    title="Comparison of Precision at K",
    width=400,
    height=300
)

chart

alt.Chart(...)

In [104]:
user_proficiency = df.groupby(["version", "isExpert"])["mse"].mean().reset_index()

#
# Grouped bar chart, sort versions on X axis to LC + EH, LC + ES, HC + EH, HC + ES
chart = alt.Chart(user_proficiency).mark_bar().encode(
    x=alt.X("version", sort=["LC + EH", "LC + ES", "HC + EH", "HC + ES"]),
    y="mse",
    color="isExpert:N",
    xOffset="isExpert:N",
    tooltip=["version", "isExpert", "mse"]
).properties(
    title="",
    width=400,
    height=300
)

chart

alt.Chart(...)

In [106]:

duration_data = df.groupby("version")["duration"].mean().reset_index()

# Bar chart
chart = alt.Chart(duration_data).mark_bar().encode(
    x=alt.X("version", sort=["LC + EH", "LC + ES", "HC + EH", "HC + ES"]),
    y="duration",
    color="version",
    tooltip=["version", "duration"]
).properties(
    title="Duration Analysis",
    width=400,
    height=300
)

chart

alt.Chart(...)

In [107]:
# Data preparation
mse_data = df.groupby(["version", "isExpert"])["mse"].mean().reset_index()

# Grouped bar chart
chart = alt.Chart(mse_data).mark_bar().encode(
    x=alt.X("version", sort=["LC + EH", "LC + ES", "HC + EH", "HC + ES"]),
    y="mse",
    color="isExpert:N",
    column="isExpert:N",
    tooltip=["version", "isExpert", "mse"]
).properties(
    title="MSE Comparison",
    width=400,
    height=300
)

chart

alt.Chart(...)

In [116]:
# open csv file
dialogCount = pd.read_csv('countDialogOpen.csv')


#join df with dialogCount by matching userid
joined = df.merge(dialogCount[['userid','count']], on='userid', how='left')

#drop rows with too many dialog boxes opened

joined = joined[joined['count'] < 80]


# joined

#graph that compares the number of dialog boxes opened with the mse and version
plot = alt.Chart(joined).mark_circle().encode(
    x='count',
    y='mse',
    color='version:N',
    column=alt.Column('version:N', sort=['LC + EH', 'LC + ES', 'HC + EH', 'HC + ES']),
).properties(
    title='Number of Dialog Boxes Opened vs MSE vs Version'
)
plot


alt.Chart(...)

In [117]:
#calculate average mse, pak and src per version per expertise (expert or not)
df['avg_mse'] = df.groupby(['version','isExpert'])['mse'].transform('mean')
df['avg_pak'] = df.groupby(['version','isExpert'])['pak'].transform('mean')
df['avg_src'] = df.groupby(['version','isExpert'])['src'].transform('mean')

dfa = df[['version','isExpert','avg_mse','avg_pak','avg_src']].drop_duplicates()

#plot average pak, src and mse per version (expert or not) in barchart

# Chart 1: Average MSE for each version
chart1 = alt.Chart(dfa).mark_bar().encode(
    x=alt.X('version', sort=['LC + EH', 'LC + ES', 'HC + EH', 'HC + ES']),
    y='avg_mse:Q',
    color='isExpert',
    xOffset='isExpert:N'
).properties(
    title='Average Mean Squared Error (MSE) for Each Version'
)

# Chart 2: Average Duration for each version
chart2 = alt.Chart(dfa).mark_bar().encode(
    x=alt.X('version', sort=['LC + EH', 'LC + ES', 'HC + EH', 'HC + ES']),
    y='avg_pak:Q',
    color='isExpert',
    xOffset='isExpert:N'
).properties(
    title='Average Precision at 6 for Each Version'
)

# Chart 3: Average Duration for each version
chart3 = alt.Chart(dfa).mark_bar().encode(
    x=alt.X('version', sort=['LC + EH', 'LC + ES', 'HC + EH', 'HC + ES']),
    y='avg_src:Q',
    color='isExpert',
    xOffset='isExpert:N'
).properties(
    title='Average Spearman rank Coefficient (SRC) for Each Version'
)

chart1 | chart2 | chart3


alt.HConcatChart(...)

In [118]:
#shows the impact of the weights on the accuracy

pose  = alt.Chart(df).mark_circle().encode(
    x='poseweight',
    y='mse',
    tooltip=['version', 'mse', 'poseweight'],
).properties(
    title='Pose Weight vs MSE vs Version'
)

color = alt.Chart(df).mark_circle().encode(
    x='colorweight',
    y='mse',
    tooltip=['version', 'mse', 'colorweight']
).properties(
    title='Color Weight vs MSE vs Version'
)

object = alt.Chart(df).mark_circle().encode(
    x='objectweight',
    y='mse',
    tooltip=['version', 'mse', 'objectweight']
).properties(
    title='Object Weight vs MSE vs Version'
)

pose | color | object


#combine the three charts into one each


alt.HConcatChart(...)

In [22]:
#show most popular axis (pose, color, object)

#remove rows with nan
df = df.dropna()

pose = alt.Chart(df).mark_bar().encode(
    x=alt.X('poseweight', bin=alt.Bin(maxbins=10)),
    y='count()',
).properties(
    title='Pose Weight vs Count vs Version'
)

color = alt.Chart(df).mark_bar().encode(
    x=alt.X('colorweight', bin=alt.Bin(maxbins=10)),
    y='count()',
    tooltip=['version', 'colorweight']
).properties(
    title='Color Weight vs Count vs Version'
)

object = alt.Chart(df).mark_bar().encode(
    x=alt.X('objectweight', bin=alt.Bin(maxbins=10)),
    y='count()',
    tooltip=['version', 'objectweight']
).properties(
    title='Object Weight vs Count vs Version'
)

(pose | color | object).save('charts/weights_popularity.png')

pose | color | object

alt.HConcatChart(...)

In [121]:
#See if complexity of query affects the accuracy

df_sum = df

df_sum['sum'] = df_sum['poseweight'] + df_sum['colorweight'] + df_sum['objectweight']

mse = alt.Chart(df_sum).mark_circle().encode(
    x='sum',
    y='mse',
    tooltip=['version', 'mse', 'sum']
).properties(
    title='Sum of Weights vs MSE'
)


pak = alt.Chart(df_sum).mark_circle().encode(
    x='sum',
    y='pak',
    tooltip=['version', 'pak', 'sum']
).properties(
    title='Sum of Weights vs PAK'
)

src = alt.Chart(df_sum).mark_circle().encode(
    x='sum',
    y='src',
    tooltip=['version', 'src', 'sum']
).properties(
    title='Sum of Weights vs SRC'
)

mse | pak | src


alt.HConcatChart(...)

In [122]:
#calculate average mse per user

x = df.groupby(['userid', 'isExpert', 'version'])['mse'].mean().reset_index()

#load csv file
dialogCount = pd.read_csv('countDialogOpen.csv')

#join df with dialogCount by matching userid

joined = x.merge(dialogCount[['userid','count']], on='userid', how='left')




joined_avg = joined.groupby(['version', 'isExpert'])['mse'].mean().reset_index()

#chart

chart = alt.Chart(joined_avg).mark_bar().encode(
    x=alt.X("version", sort=["LC + EH", "LC + ES", "HC + EH", "HC + ES"]),
    y="mse",
    color="isExpert:N",
    column="isExpert:N",
    tooltip=["version", "isExpert", "mse"]
).properties(
    title="MSE Comparison",
    width=400,
    height=300
)
chart

alt.Chart(...)

In [85]:
joined_avg

,version,isExpert,mse
0,HC + EH,Expert,238.104167
1,HC + EH,Non-Expert,313.880952
2,HC + ES,Expert,179.444444
3,HC + ES,Non-Expert,84.305556
4,LC + EH,Expert,287.916667
5,LC + EH,Non-Expert,425.902778
6,LC + ES,Expert,234.125000
7,LC + ES,Non-Expert,194.920635
